In [37]:
import os
import sys
from typing import List, Dict

import lightgbm as lgb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

c:\Users\com\Documents\GitHub\level1-classificationinmachinelearning-recsys-06


In [38]:
# 파일 호출
data_path: str = "../../data"
## raw.csv가 없는 경우 실행
# from Code.dataset.merge_all import merge_all
# df = merge_all(data_path)
df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "after_eda.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))  # ID, target 열만 가진 데이터 미리 호출


In [22]:
df

,ID,_type,target,difference,percent,hashrate,taker_buy_sell_ratio,open_interest,funding_rates,coinbase_premium_index,utxo_count,transactions_count_total,total_liquidations,liquidation_diff,total_taker_volume,estimated_block_reward_usd
0,2023-01-01 00:00:00,train,2.0,20.388568,0.123292,5.062917e+11,0.904774,6.271344e+09,0.005049,-0.059650,83308092.0,11457.0,0.012000,0.012000,9.785846e+07,104057.980295
1,2023-01-01 01:00:00,train,1.0,-8.986731,-0.054277,1.687639e+11,1.655721,6.288683e+09,0.005049,-0.053047,83314883.0,5832.0,0.712000,-0.712000,6.376728e+07,104403.922724
2,2023-01-01 02:00:00,train,1.0,-14.516930,-0.087725,3.375278e+11,1.027512,6.286796e+09,0.005049,-0.057952,83314090.0,5550.0,0.000000,0.000000,4.746397e+07,104021.263731
3,2023-01-01 03:00:00,train,1.0,-8.920715,-0.053955,2.109549e+11,0.874477,6.284575e+09,0.005067,-0.058912,83326258.0,5245.0,0.593000,0.593000,6.130988e+07,103993.945406
4,2023-01-01 04:00:00,train,2.0,4.695906,0.028417,2.953368e+11,0.966796,6.291582e+09,0.006210,-0.061373,83339168.0,6942.0,0.361000,0.361000,6.281486e+07,104100.262482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,test,NaN,NaN,0.000000,3.153372e+11,NaN,1.486836e+10,NaN,-0.002376,179820708.0,10048.0,0.953500,0.466500,NaN,251543.725340
11548,2024-04-26 04:00:00,test,NaN,NaN,0.000000,6.306744e+11,NaN,NaN,NaN,-0.018268,179833897.0,NaN,6.723208,6.431208,NaN,244385.395493
11549,2024-04-26 05:00:00,test,NaN,NaN,0.000000,6.306744e+11,NaN,NaN,NaN,0.002866,179851249.0,NaN,7.013653,-3.419327,NaN,241170.877304
11550,2024-04-26 06:00:00,test,NaN,NaN,0.000000,4.204496e+11,NaN,NaN,NaN,-0.003184,179852452.0,NaN,2.459000,-0.853000,NaN,232681.919923


In [39]:
eda_df = df.drop(columns=['difference', 'percent'])

In [40]:
import yaml

config_path ='../../config-sample.yaml'

with open(config_path,'r') as file:
    config = yaml.safe_load(file)

In [30]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# _type에 따라 train, test 분리
train_df = eda_df.loc[eda_df["_type"] == "train"].drop(columns=["_type"])
test_df = eda_df.loc[eda_df["_type"] == "test"].drop(columns=["_type"])

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)

# XGBoost 모델을 위한 DMatrix로 변환
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

# XGBoost 모델 학습 파라미터
params = config["xgboost"]
params["objective"] = "multi:softprob"  # 다중 클래스 분류를 위한 설정
params["num_class"] = len(y_train.unique())  # 클래스 개수
params["eval_metric"] = "mlogloss"  # 손실 함수 (멀티클래스 로지스틱 손실)

# 학습 시 평가 데이터로 검증 셋 사용
evals = [(dtrain, "train"), (dvalid, "eval")]

# XGBoost 모델 훈련
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100,  # 트리의 개수
    evals=evals,
    early_stopping_rounds=10  # 조기 중단
)

# 예측 (훈련 데이터셋)
y_train_pred_prob = xgb_model.predict(dtrain)
y_train_pred = y_train_pred_prob.argmax(axis=1)

# 예측 (검증 데이터셋)
y_valid_pred_prob = xgb_model.predict(dvalid)
y_valid_pred = y_valid_pred_prob.argmax(axis=1)

# 성능 평가 (훈련 데이터셋)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_classification_rep = classification_report(y_train, y_train_pred)

# 성능 평가 (검증 데이터셋)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_classification_rep = classification_report(y_valid, y_valid_pred)

print(f"훈련 정확도: {train_accuracy}")
print(train_classification_rep)
print(f"검증 정확도: {valid_accuracy}")
print(valid_classification_rep)

[0]	train-mlogloss:1.36453	eval-mlogloss:1.37054
[1]	train-mlogloss:1.34343	eval-mlogloss:1.35522
[2]	train-mlogloss:1.32313	eval-mlogloss:1.34137
[3]	train-mlogloss:1.30381	eval-mlogloss:1.32820
[4]	train-mlogloss:1.28554	eval-mlogloss:1.31624


c:\Users\com\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:05:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "early_stopping_rounds", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[5]	train-mlogloss:1.26893	eval-mlogloss:1.30495
[6]	train-mlogloss:1.25297	eval-mlogloss:1.29475
[7]	train-mlogloss:1.23807	eval-mlogloss:1.28491
[8]	train-mlogloss:1.22386	eval-mlogloss:1.27594
[9]	train-mlogloss:1.21011	eval-mlogloss:1.26733
[10]	train-mlogloss:1.19718	eval-mlogloss:1.25961
[11]	train-mlogloss:1.18443	eval-mlogloss:1.25212
[12]	train-mlogloss:1.17251	eval-mlogloss:1.24473
[13]	train-mlogloss:1.16086	eval-mlogloss:1.23767
[14]	train-mlogloss:1.14903	eval-mlogloss:1.23115
[15]	train-mlogloss:1.13817	eval-mlogloss:1.22512
[16]	train-mlogloss:1.12787	eval-mlogloss:1.21924
[17]	train-mlogloss:1.11782	eval-mlogloss:1.21398
[18]	train-mlogloss:1.10834	eval-mlogloss:1.20910
[19]	train-mlogloss:1.09872	eval-mlogloss:1.20401
[20]	train-mlogloss:1.08960	eval-mlogloss:1.19938
[21]	train-mlogloss:1.08097	eval-mlogloss:1.19524
[22]	train-mlogloss:1.07217	eval-mlogloss:1.19099
[23]	train-mlogloss:1.06381	eval-mlogloss:1.18747
[24]	train-mlogloss:1.05619	eval-mlogloss:1.18421
[25]	

In [32]:
# 테스트 데이터셋을 DMatrix로 변환
dtest = xgb.DMatrix(test_df.drop(["target", "ID"], axis=1))

# 테스트 데이터에 대한 예측 수행
y_test_pred_prob = xgb_model.predict(dtest)

# 다중 클래스 예측일 경우 가장 높은 확률을 가진 클래스를 예측값으로 변환
y_test_pred = y_test_pred_prob.argmax(axis=1)

# 예측 결과 출력
y_test_pred


array([1, 1, 2, ..., 1, 1, 1], dtype=int64)

In [33]:
submission_df = submission_df.assign(target=pd.DataFrame(y_test_pred))
submission_df

,ID,target
0,2024-01-01 00:00:00,1
1,2024-01-01 01:00:00,1
2,2024-01-01 02:00:00,2
3,2024-01-01 03:00:00,2
4,2024-01-01 04:00:00,2
...,...,...
2787,2024-04-26 03:00:00,3
2788,2024-04-26 04:00:00,1
2789,2024-04-26 05:00:00,1
2790,2024-04-26 06:00:00,1


In [34]:
submission_df['target'].value_counts()

target
2    1619
1    1058
3      73
0      42
Name: count, dtype: int64

In [36]:
submission_df.to_csv("XGBoost_after_eda_raw.csv",index=False)

In [35]:
import plotly.express as px

# Feature importance 평가
importance = model.get_score(importance_type='weight')
importance_df = pd.DataFrame({
    'Feature': [k for k in importance.keys()],
    'Importance': importance.values()
}).sort_values(by='Importance', ascending=False)

# 시각화
fig = px.bar(importance_df, x='Feature', y='Importance', title='Feature Importance')
fig.show()

### hashrate_diff 추가

In [41]:
eda_df['hashrate_diff'] = eda_df['hashrate'].diff()

In [42]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# _type에 따라 train, test 분리
train_df = eda_df.loc[eda_df["_type"] == "train"].drop(columns=["_type"])
test_df = eda_df.loc[eda_df["_type"] == "test"].drop(columns=["_type"])

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)

# XGBoost 모델을 위한 DMatrix로 변환
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

# XGBoost 모델 학습 파라미터
params = config["xgboost"]
params["objective"] = "multi:softprob"  # 다중 클래스 분류를 위한 설정
params["num_class"] = len(y_train.unique())  # 클래스 개수
params["eval_metric"] = "mlogloss"  # 손실 함수 (멀티클래스 로지스틱 손실)

# 학습 시 평가 데이터로 검증 셋 사용
evals = [(dtrain, "train"), (dvalid, "eval")]

# XGBoost 모델 훈련
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100,  # 트리의 개수
    evals=evals,
    early_stopping_rounds=10  # 조기 중단
)

# 예측 (훈련 데이터셋)
y_train_pred_prob = xgb_model.predict(dtrain)
y_train_pred = y_train_pred_prob.argmax(axis=1)

# 예측 (검증 데이터셋)
y_valid_pred_prob = xgb_model.predict(dvalid)
y_valid_pred = y_valid_pred_prob.argmax(axis=1)

# 성능 평가 (훈련 데이터셋)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_classification_rep = classification_report(y_train, y_train_pred)

# 성능 평가 (검증 데이터셋)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_classification_rep = classification_report(y_valid, y_valid_pred)

print(f"훈련 정확도: {train_accuracy}")
print(train_classification_rep)
print(f"검증 정확도: {valid_accuracy}")
print(valid_classification_rep)

[0]	train-mlogloss:1.36414	eval-mlogloss:1.37045
[1]	train-mlogloss:1.34270	eval-mlogloss:1.35579
[2]	train-mlogloss:1.32253	eval-mlogloss:1.34182
[3]	train-mlogloss:1.30371	eval-mlogloss:1.32880
[4]	train-mlogloss:1.28599	eval-mlogloss:1.31665
[5]	train-mlogloss:1.26882	eval-mlogloss:1.30550
[6]	train-mlogloss:1.25301	eval-mlogloss:1.29479


c:\Users\com\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning:

[23:40:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "early_stopping_rounds", "verbose" } are not used.




[7]	train-mlogloss:1.23771	eval-mlogloss:1.28477
[8]	train-mlogloss:1.22286	eval-mlogloss:1.27571
[9]	train-mlogloss:1.20842	eval-mlogloss:1.26658
[10]	train-mlogloss:1.19483	eval-mlogloss:1.25829
[11]	train-mlogloss:1.18200	eval-mlogloss:1.25029
[12]	train-mlogloss:1.16932	eval-mlogloss:1.24315
[13]	train-mlogloss:1.15746	eval-mlogloss:1.23675
[14]	train-mlogloss:1.14614	eval-mlogloss:1.23039
[15]	train-mlogloss:1.13486	eval-mlogloss:1.22438
[16]	train-mlogloss:1.12421	eval-mlogloss:1.21844
[17]	train-mlogloss:1.11425	eval-mlogloss:1.21306
[18]	train-mlogloss:1.10501	eval-mlogloss:1.20806
[19]	train-mlogloss:1.09559	eval-mlogloss:1.20338
[20]	train-mlogloss:1.08660	eval-mlogloss:1.19898
[21]	train-mlogloss:1.07762	eval-mlogloss:1.19500
[22]	train-mlogloss:1.06899	eval-mlogloss:1.19097
[23]	train-mlogloss:1.06058	eval-mlogloss:1.18788
[24]	train-mlogloss:1.05264	eval-mlogloss:1.18488
[25]	train-mlogloss:1.04444	eval-mlogloss:1.18151
[26]	train-mlogloss:1.03694	eval-mlogloss:1.17899
[27